<h1 style="text-align: center; font-family: Verdana; font-size: 16px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: #FF1493; background-color: #ffffff;">Bristol-Myers Squibb – Molecular Translation</h1>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Getting Data 

In [ ]:
 ! pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!mkdir /content/data
!cd /content/data

In [ ]:
!kaggle competitions download -c bms-molecular-translation
!kaggle competitions list -s bms-molecular-translation
!wget https://www.kaggle.com/c/bms-molecular-translation/data#:~:text=get_app-,Download,-All
!kaggle datasets download -d dschettler8845/automl-efficientdet-efficientnetv2
!kaggle datasets download -d dschettler8845/bms-csvs-w-extra-metadata
!kaggle datasets download -d dschettler8845/bms-test-dataset-192x384
!kaggle datasets download -d dschettler8845/bms-train-tfrecords-half-length
!kaggle datasets download -d kedarvaidya05/bms-efficientnetv2-tpu-e2e-pipeline-in-3hrs

In [ ]:
!cp /content/bms-train-tfrecords-half-length.zip  /content/drive/MyDrive/Molecular_translation

In [ ]:
!mkdir /content/drive/MyDrive/Molecular_translation/bms-molecular-translation
!mkdir /content/drive/MyDrive/Molecular_translation/automl-efficientdet-efficientnetv2
!mkdir /content/drive/MyDrive/Molecular_translation/bms-csvs-w-extra-metadata
!mkdir /content/drive/MyDrive/Molecular_translation/bms-test-dataset-192x384
!mkdir /content/drive/MyDrive/Molecular_translation/bms-train-tfrecords-half-length
!mkdir /content/drive/MyDrive/Molecular_translation/bms-efficientnetv2-tpu-e2e-pipeline-in-3hrs

In [ ]:
!unzip /content/drive/MyDrive/Molecular_translation/bms-train-tfrecords-half-length.zip -d /content/drive/MyDrive/Molecular_translation/bms-train-tfrecords-half-length

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Molecular_translation"

# IMPORTS

In [ ]:
!pip install --user tensorflow-addons==0.8.3
!pip install --user tensorflow==2.2.0-rc3
!pip install kaggle_datasets

In [ ]:
!pip install python-levenshtein

     |████████████████████████████████| 50 kB 2.8 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149867 sha256=4de3c2b904dc996c9618ae07da4758462010ef7d06a72268a00c42ca0968806a
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-levenshtein


In [ ]:
# Installs
print("\n... PIP/APT INSTALLS STARTING ...\n")
# Pips
!pip install -q --upgrade pip
!pip install -q pydot
!pip install -q pydotplus

# Apt-get
!apt-get install -q graphviz
print("\n... PIP/APT INSTALLS COMPLETE ...\n")

print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");

# Library used to easily calculate LD
import Levenshtein

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import imageio
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2


def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    
print("\n\n... IMPORTS COMPLETE ...\n")
    
print("\n... SEEDING FOR DETERMINISTIC BEHAVIOUR ...\n")
seed_it_all()

# SETUP

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.1 ACCELERATOR DETECTION</h3>



In [ ]:
print(f"\n... ACCELERATOR SETUP STARTING ...\n")


try:
    
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
except ValueError:
    TPU = None

if TPU:
    print(f"\n... RUNNING ON TPU - {TPU.master()}...")
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    print(f"\n... RUNNING ON CPU/GPU ...")
    
    strategy = tf.distribute.get_strategy() 


N_REPLICAS = strategy.num_replicas_in_sync
    
print(f"... # OF REPLICAS: {N_REPLICAS} ...\n")

print(f"\n... ACCELERATOR SETUP COMPLTED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.2 COMPETITION DATA ACCESS</h3>

In [ ]:
print("DATA ACCESS SETUP STARTED")

if TPU:
    
    DATA_DIR = KaggleDatasets().get_gcs_path('bms-train-tfrecords-half-length')
    TEST_DATA_DIR = KaggleDatasets().get_gcs_path('bms-test-dataset-192x384')
else:
    # Local path to training and validation images
    DATA_DIR = f"{DATA_DIR}/bms-train-tfrecords-half-length"
    TEST_DATA_DIR = f"{DATA_DIR}/bms-test-dataset-192x384"
    
print(f"\n... DATA DIRECTORY PATH IS:\n\t--> {DATA_DIR}")
print(f"... TEST DATA DIRECTORY PATH IS:\n\t--> {TEST_DATA_DIR}")

print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

print(f"... IMMEDIATE CONTENTS OF TESTT DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(TEST_DATA_DIR, "*")): print(f"\t--> {file}")

  

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.3 LEVERAGING MIXED PRECISION</h3>

---



In [ ]:
print(f"\n... MIXED PRECISION SETUP STARTING ...\n")
print("\n... SET TF TO OPERATE IN MIXED PRECISION – `bfloat16` – IF ON TPU ...")

tf.keras.mixed_precision.set_global_policy('mixed_bfloat16' if TPU else 'float32')

# target data type, bfloat16 when using TPU to improve throughput
TARGET_DTYPE = tf.bfloat16 if TPU else tf.float32
print(f"\t--> THE TARGET DTYPE HAS BEEN SET TO {TARGET_DTYPE} ...")

# The policy specifies two important aspects of a layer: 
#     1. The dtype the layer's computations are done in
#     2. The dtype of a layer's variables. 
print(f"\n... TWO IMPORTANT ASPECTS OF THE GLOBAL MIXED PRECISION POLICY:")
print(f'\t--> COMPUTE DTYPE  : {tf.keras.mixed_precision.global_policy().compute_dtype}')
print(f'\t--> VARIABLE DTYPE : {tf.keras.mixed_precision.global_policy().variable_dtype}')

print(f"\n\n... MIXED PRECISION SETUP COMPLTED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.4 LEVERAGING XLA OPTIMIZATIONS</h3>

---


In [ ]:
print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.5 BASIC DATA DEFINITIONS & INITIALIZATIONS</h3>

---


In [ ]:
print("\n... BASIC DATA SETUP STARTING ...\n")

# All the possible tokens in our InChI 'language'
TOKEN_LIST = ["<PAD>", "InChI=1S/", "<END>", "/c", "/h", "/m", "/t", "/b", "/s", "/i"] +\
             ['Si', 'Br', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'C', 'H', 'B', ] +\
             [str(i) for i in range(167,-1,-1)] +\
             ["\+", "\(", "\)", "\-", ",", "D", "T"]
print(f"\n... TOKEN LIST:")
for i, tok in enumerate(TOKEN_LIST): print(f"\t--> INTEGER-IDX = {i:<3}  –––  STRING = {tok}")

START_TOKEN = tf.constant(TOKEN_LIST.index("InChI=1S/"), dtype=tf.uint8)
END_TOKEN = tf.constant(TOKEN_LIST.index("<END>"), dtype=tf.uint8)
PAD_TOKEN = tf.constant(TOKEN_LIST.index("<PAD>"), dtype=tf.uint8)

PREFIX_ORDERING = "chbtmsihtm"
print(f"\n... PREFIX ORDERING IS {PREFIX_ORDERING} ...")

# Paths to Respective Image Directories
TRAIN_DIR = os.path.join(DATA_DIR, "train_records")
VAL_DIR = os.path.join(DATA_DIR, "val_records")
TEST_DIR = os.path.join(TEST_DATA_DIR, "test_records")

# Get the Full Paths to The Individual TFRecord Files
TRAIN_TFREC_PATHS = sorted(
    tf.io.gfile.glob(os.path.join(TRAIN_DIR, "*.tfrec")), 
    key=lambda x: int(x.rsplit("_", 2)[1]))
VAL_TFREC_PATHS = sorted(
    tf.io.gfile.glob(os.path.join(VAL_DIR, "*.tfrec")), 
    key=lambda x: int(x.rsplit("_", 2)[1]))
TEST_TFREC_PATHS = sorted(
    tf.io.gfile.glob(os.path.join(TEST_DIR, "*.tfrec")), 
    key=lambda x: int(x.rsplit("_", 2)[1]))

print(f"\n... TFRECORD INFORMATION:")
for SPLIT, TFREC_PATHS in zip(["TRAIN", "VAL", "TEST"], [TRAIN_TFREC_PATHS, VAL_TFREC_PATHS, TEST_TFREC_PATHS]):
    print(f"\t--> {len(TFREC_PATHS):<3} {SPLIT:<5} TFRECORDS")

# Paths to relevant CSV files containing training and submission information
TRAIN_CSV_PATH = os.path.join(DATA_DIR, "bms-csvs-w-extra-metadata", "train_labels_w_extra.csv")
SS_CSV_PATH    = os.path.join(DATA_DIR, "bms-csvs-w-extra-metadata", "sample_submission_w_extra.csv")
print(f"\n... PATHS TO CSVS:")
print(f"\t--> TRAIN CSV: {TRAIN_CSV_PATH}")
print(f"\t--> SS CSV   : {SS_CSV_PATH}")

# When debug is true we use a smaller batch size and smaller model
DEBUG=False

print("\n\n... BASIC DATA SETUP COMPLETED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.6 INITIAL DATAFRAME INSTANTIATION</h3>

---


In [ ]:
print("\n... INITIAL DATAFRAME INSTANTIATION STARTING ...\n")

# Load the train and submission dataframes
train_df = pd.read_csv(TRAIN_CSV_PATH)
ss_df    = pd.read_csv(SS_CSV_PATH)

# --- Distribution Information ---
N_EX    = len(train_df)
N_TEST  = len(ss_df)
N_VAL   = 80_000 # Fixed from dataset creation information
N_TRAIN = N_EX-N_VAL

# --- Batching Information ---
DEBUG=False
BATCH_SIZE_DEBUG   = 2
REPLICA_BATCH_SIZE = 128 # Could probably be 128

if DEBUG:
    REPLICA_BATCH_SIZE = BATCH_SIZE_DEBUG
OVERALL_BATCH_SIZE = REPLICA_BATCH_SIZE*N_REPLICAS


# --- Input Image Information ---
IMG_SHAPE = (192,384,3)

# --- Autocalculate Training/Validation/Testing Information ---
TRAIN_STEPS = N_TRAIN  // OVERALL_BATCH_SIZE
VAL_STEPS   = N_VAL    // OVERALL_BATCH_SIZE
TEST_STEPS  = int(np.ceil(N_TEST/OVERALL_BATCH_SIZE))

# This is for padding our test dataset so we only have whole batches
REQUIRED_DATASET_PAD = OVERALL_BATCH_SIZE-N_TEST%OVERALL_BATCH_SIZE

# --- Modelling Information ---
ATTN_EMB_DIM  = 192
N_RNN_UNITS   = 512

print(f"\n... # OF TRAIN+VAL EXAMPLES  : {N_EX:<7} ...")
print(f"... # OF TRAIN EXAMPLES      : {N_TRAIN:<7} ...")
print(f"... # OF VALIDATION EXAMPLES : {N_VAL:<7} ...")
print(f"... # OF TEST EXAMPLES       : {N_TEST:<7} ...\n")

print(f"\n... REPLICA BATCH SIZE    : {REPLICA_BATCH_SIZE} ...")
print(f"... OVERALL BATCH SIZE    : {OVERALL_BATCH_SIZE} ...\n")

print(f"\n... IMAGE SHAPE           : {IMG_SHAPE} ...\n")

print(f"\n... TRAIN STEPS PER EPOCH : {TRAIN_STEPS:<5} ...")
print(f"... VAL STEPS PER EPOCH   : {VAL_STEPS:<5} ...")
print(f"... TEST STEPS PER EPOCH  : {TEST_STEPS:<5} ...\n")

print("\n... TRAIN DATAFRAME ...\n")
display(train_df.head(3))

print("\n... SUBMISSION DATAFRAME ...\n")
display(ss_df.head(3))

print("\n... INITIAL DATAFRAME INSTANTIATION COMPLETED...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.7 USER INPUT VARIABLES</h3>

---


In [ ]:
print("\n... SPECIAL VARIABLE SETUP STARTING ...\n")


# Whether to start training using previously checkpointed model
LOAD_MODEL        = False
ENCODER_CKPT_PATH = ""
TRANSFORMER_CKPT_PATH = ""

if LOAD_MODEL:
    if TRANSFORMER_CKPT_PATH != "":
        print(f"... TRANSFORMER MODEL TRAINING WILL RESUME FROM PREVIOUS CHECKPOINT:\n\t-->{TRANSFORMER_CKPT_PATH}\n")
    elif ENCODER_CKPT_PATH != "":
        print(f"\n... ENCODER MODEL TRAINING WILL RESUME FROM PREVIOUS CHECKPOINT:\n\t-->{ENCODER_CKPT_PATH}\n")    
    else:
        print(f"\n... MODEL TRAINING WILL START FROM SCRATCH ...\n")
else:
    print(f"\n... MODEL TRAINING WILL START FROM SCRATCH ...\n")

    
print("\n... SPECIAL VARIABLE SETUP COMPLETED ...\n")

#HELPER FUNCTION & CLASSES

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">3.1 GENERAL HELPER FUNCTIONS</h3>

---

In [ ]:
def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]


def tf_load_image(path, img_size=(192,384,3), invert=False):
    
    img = decode_img(tf.io.read_file(path), img_size, n_channels=3, invert=invert)        
    return img
    
    
def decode_image(image_data, resize_to=(192,384,3)):
    
    image = tf.image.decode_png(image_data, channels=3)
    image = tf.reshape(image, resize_to)
    return tf.cast(image, TARGET_DTYPE)
    
    
# sparse tensors are required to compute the Levenshtein distance
def dense_to_sparse(dense):
    
    indices = tf.where(tf.ones_like(dense))
    values = tf.reshape(dense, (MAX_LEN*OVERALL_BATCH_SIZE,))
    sparse = tf.SparseTensor(indices, values, dense.shape)
    return sparse

def get_levenshtein_distance(preds, lbls):
    
    preds = tf.where(tf.not_equal(lbls, END_TOKEN) & tf.not_equal(lbls, PAD_TOKEN), preds, 0)
    lbls = tf.where(tf.not_equal(lbls, END_TOKEN), lbls, 0)

    preds_sparse = dense_to_sparse(preds)
    lbls_sparse = dense_to_sparse(lbls)

    batch_distance = tf.edit_distance(preds_sparse, lbls_sparse, normalize=False)
    mean_distance = tf.math.reduce_mean(batch_distance)
    
    return mean_distance

# PREPARE THE DATASET

In [ ]:
print("\n\n... STARTING PREPARING VARIABLES FOR DATASET ...\n")

tok_2_int = {c.strip("\\"):i for i,c in enumerate(TOKEN_LIST)}
int_2_tok = {v:k for k,v in tok_2_int.items()}

# Max Length Was Determined Previously Using... 
#     >>> MAX_LEN = train_df.InChI.progress_apply(lambda x: len(re.findall("|".join(TOKEN_LIST), x))).max()+1
MAX_LEN = ((train_df.inchi_token_len.max()+1)//2) # //2 yields 138... which is half of max length (speeds up training)
VOCAB_LEN = len(int_2_tok)

print(f"\t--> TOKEN TO INTEGER MAP     : {tok_2_int}")
print(f"\t--> INTEGER TO TOKEN MAP     : {int_2_tok}")
print(f"\t--> MAX # OF TOKENS IN INCHI : {MAX_LEN}")
print(f"\t--> LENGTH OF VOCAB          : {VOCAB_LEN}")

print(f"\n\n\t--> CONVERTED INCHI STRINGS  :")
for i, row in train_df.iloc[:N_VAL].sample(3).iterrows():
    print(f"\n\t\t--> EXAMPLE #{i} FROM THE VALIDATION DATASET")
    print("\t\t\t--> RAW INCHI : ", row["InChI"])

print("\n\n... PREPARING VARIABLES FOR DATASET COMPLETED ...\n")

In [ ]:
def decode(serialized_example, is_test=False, tokenized_inchi=True):
    
    feature_dict = {
        'image': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=''),
    }
    
    if not is_test:
        if tokenized_inchi:
            feature_dict["inchi"] = tf.io.FixedLenFeature(shape=[MAX_LEN], dtype=tf.int64, default_value=[0]*MAX_LEN)
        else:
            feature_dict["inchi"] = tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value='')
    else:
        feature_dict["image_id"] = tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value='')
    
    # Define a parser
    features = tf.io.parse_single_example(serialized_example, features=feature_dict)
    
    # Decode the tf.string
    image = decode_image(features['image'], resize_to=IMG_SHAPE)
    
    # Figure out the correct information to return
    if is_test:
        image_id = features["image_id"] 
        return image, image_id
    else:
        if tokenized_inchi:
            target = tf.cast(features["inchi"], tf.uint8)
        else:
            target = features["inchi"]
        return image, target

In [ ]:
print("\n... DECODING RAW TFRECORD DATASETS STARTING ...\n")

# Decode the tfrecords completely –– decode is our `_parse_function` (from recipe above)
train_ds = raw_train_ds.map(lambda x: decode(x, is_test=False))
val_ds = raw_val_ds.map(lambda x: decode(x, is_test=False))
test_ds = raw_test_ds.map(lambda x: decode(x, is_test=True))

print(f"\n... THE DECODED TF.DATA.TFRECORDDATASET OBJECT:" \
      f"\n\t--> ((image), (image_id - optional), (inchi))" \
      f"\n\t--> {train_ds}\n")

print("\n... 2 EXAMPLES OF IMAGES AND LABELS AFTER DECODING ...")
for i, (img, inchi) in enumerate(train_ds.take(2)):
    print(f"\nIMAGE SHAPE : {img.shape}")
    print(f"IMAGE INCHI : {[int_2_tok[x] for x in inchi.numpy()]}\n")
    plt.figure(figsize=(10,10))
    plt.imshow(img.numpy().astype(np.int64), cmap="gray")
    plt.title(f"{''.join([int_2_tok[x] for x in inchi.numpy() if x!=0][:50])} ... [truncated]")
    plt.show()

print("\n... DECODING RAW TFRECORD DATASETS COMPLETED ...\n")

#MODEL PREPARATION

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.1 UNDERSTANDING THE MODELS - ViT</h3>

---



<h3 style="font-family: Verdana; font-size: 16px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.1.1 ViT - Implement Patch Creation as a Layer</h3>

---

In [ ]:
class PatchCreator(tf.keras.layers.Layer):
    def __init__(self, patch_size):
        super(PatchCreator, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

    

demo_img, demo_lbl = next(iter(train_ds.unbatch().batch(1)))
demo_patch_size=16


with tf.device('/CPU:0'):
    patch_creator = PatchCreator(demo_patch_size)
    patches = patch_creator(demo_img)

print(f"Image size: {IMG_SHAPE}")
print(f"Patch size: {(demo_patch_size, demo_patch_size)}")
print(f"Patches shape: {patches.shape}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")


print("\n\n... ORIGINAL IMAGE ...\n")
plt.figure(figsize=(18,9))
plt.imshow(demo_img[0].numpy().astype("float32")/255.)
plt.axis('off')
plt.tight_layout()
plt.show()


print("\n\n\n... IMAGE PATCHES ...\n")
plt.figure(figsize=(18,9))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(int(np.ceil(IMG_SHAPE[0]/demo_patch_size)), int(np.ceil(IMG_SHAPE[1]/demo_patch_size)), i + 1)
    patch_img = tf.reshape(patch, (demo_patch_size, demo_patch_size, 3))
    plt.imshow(patch_img.numpy().astype("float32")/255.)
    plt.axis("off")
plt.tight_layout()
plt.subplots_adjust(wspace=0.03, hspace=0.06)
plt.show()

In [ ]:
tf.range(start=0, limit=288, delta=1)

In [ ]:
class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        
        self.dense_projection = tf.keras.layers.Dense(units=self.projection_dim)
        self.positions = tf.reshape(tf.range(start=0, limit=self.num_patches, delta=1), (self.num_patches,))
        self.position_embedding = tf.keras.layers.Embedding(input_dim=self.num_patches, 
                                                            output_dim=self.projection_dim)

    def call(self, patch):
        encoded = self.dense_projection(patch) + self.position_embedding(self.positions)
        return encoded
    
demo_projection_dim = 128

with tf.device('/CPU:0'):
    patch_encoder = PatchEncoder(num_patches=patches.shape[1], projection_dim=demo_projection_dim)
    encoded_patches = patch_encoder(patches)

print(f"Number of Patches: {patches.shape[1]}")    
print(f"Patch size: {(demo_patch_size, demo_patch_size)}")
print(f"Encoded patches shape: {encoded_patches.shape}")

print("\n\n... ORIGINAL IMAGE ...\n")
plt.figure(figsize=(18,9))
plt.imshow(demo_img[0].numpy().astype("float32")/255.)
plt.axis('off')
plt.tight_layout()
plt.show()

print("\n\n\n... IMAGE PATCHES ...\n")
plt.figure(figsize=(18,9))
for i, patch in enumerate(patches[0]):
    w,h = int(np.ceil(IMG_SHAPE[0]/demo_patch_size)), int(np.ceil(IMG_SHAPE[1]/demo_patch_size))
    ax = plt.subplot(w,h,i+1)
    patch_img = tf.reshape(patch, (demo_patch_size, demo_patch_size, 3))
    plt.imshow(patch_img.numpy().astype("float32")/255.)
    plt.axis("off")
plt.tight_layout()
plt.subplots_adjust(wspace=0.03, hspace=0.06)
plt.show()

with tf.device('/CPU:0'):
    enc_patches_rescaled = patch_creator(tf.expand_dims(tf.image.resize(tf.reshape(tf.reduce_mean(encoded_patches[0], axis=-1), (w, h, 1)), (IMG_SHAPE[0], IMG_SHAPE[1])), axis=0))


print("\n\n\n... IMAGE EMBEDDING AS PATCH VISUALIZATION ...\n")
plt.figure(figsize=(18,9))
for i, enc_patch in enumerate(enc_patches_rescaled[0]):
    ax = plt.subplot(w,h,i+1)
    enc_patch_img = tf.reshape(enc_patch, (demo_patch_size, demo_patch_size))
    plt.imshow(enc_patch_img.numpy().astype("float32")/255., cmap="jet")
    plt.axis("off")
plt.tight_layout()
plt.subplots_adjust(wspace=0.03, hspace=0.06)
plt.show()

<h3 style="font-family: Verdana; font-size: 16px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.1.3 Build the ViT Model</h3>

---

The ViT model consists of multiple Transformer blocks, which use the layers.MultiHeadAttention layer as a self-attention mechanism applied to the sequence of patches. The Transformer blocks produce a **[batch_size, num_patches, projection_dim]** tensor, which is processed via an classifier head with softmax to produce the final class probabilities output.

Unlike the technique described in the paper, which prepends a learnable embedding to the sequence of encoded patches to serve as the image representation, all the outputs of the final Transformer block are reshaped with **`tf.keras.layers.Flatten()`** and used as the image representation input to the classifier head. Note that the **`tf.keras.layers.GlobalAveragePooling1D`** layer could also be used instead to aggregate the outputs of the Transformer block, especially when the number of patches and the projection dimensions are large.

In [ ]:
class ViTEncoder(tf.keras.Model):
    def __init__(self, patch_size=16, projection_dim=256, n_transformer_layers=8, n_heads=4, dropout=0.1, img_shape=IMG_SHAPE):

        super(ViTEncoder, self).__init__()
        
        self.patch_size = patch_size
        self.n_patches = tf.cast(tf.round((img_shape[0]/self.patch_size)*(img_shape[1]/self.patch_size)), tf.int32)
        self.img_shape = img_shape
        self.projection_dim = projection_dim
        self.n_transformer_layers = n_transformer_layers
        self.mlp_intermediate_units = [self.projection_dim*2, self.projection_dim,]
        self.n_heads = n_heads
        self.dropout = dropout
        
        self.patch_creator = PatchCreator(self.patch_size)
        self.patch_encoder = PatchEncoder(self.n_patches, self.projection_dim)
        self.ln_1_layer = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.ln_2_layer = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.add_1_layer = tf.keras.layers.Add()
        self.add_2_layer = tf.keras.layers.Add()
        self.mha_layer = tf.keras.layers.MultiHeadAttention(num_heads=self.n_heads, key_dim=self.projection_dim, dropout=self.dropout)
        self.intermediate_mlp_dense_layers   = [tf.keras.layers.Dense(transformer_units, activation=tf.nn.gelu) for transformer_units in self.mlp_intermediate_units]
        self.intermediate_mlp_dropout_layers = [tf.keras.layers.Dropout(self.dropout) for transformer_units in self.mlp_intermediate_units]                
        
    def call(self, x, training):

        patches = self.patch_creator(x, training=training)
        encoded = self.patch_encoder(patches, training=training)
        
        for _ in range(self.n_transformer_layers):
            
            x1 = self.ln_1_layer(encoded, training=training)
            
            
            attention_output = self.mha_layer(x1, x1, training=training)
            
            
            x2 = self.add_1_layer([attention_output, encoded], training=training)
            
            
            x3 = self.ln_2_layer(x2, training=training)
            
            
            for i in range(len(self.mlp_intermediate_units)):
                x3 = self.intermediate_mlp_dense_layers[i](x3, training=training)
                x3 = self.intermediate_mlp_dropout_layers[i](x3, training=training)
            
            
            encoded = self.add_2_layer([x3, x2], training=training)
            
        return encoded

with tf.device('/CPU:0'):
    ViT = ViTEncoder(patch_size=demo_patch_size, projection_dim=demo_projection_dim)
    demo_encoder_output= ViT(tf.ones((1, *IMG_SHAPE)))
    IMG_SEQ_LEN, IMG_EMB_DEPTH = demo_encoder_output.shape[1], demo_encoder_output.shape[2]
    
print(f"Encoder Output Shape: {demo_encoder_output.shape}")    
print(f"Output 'Sequence' Length: {IMG_SEQ_LEN}")
print(f"Output 'Sequence' Feature Depth: {IMG_EMB_DEPTH}")


print("\n\n... ORIGINAL IMAGE ...\n")
plt.figure(figsize=(18,9))
plt.imshow(demo_img[0].numpy().astype("float32")/255.)
plt.axis('off')
plt.tight_layout()
plt.show()

print("\n\n\n... IMAGE PATCHES ...\n")
plt.figure(figsize=(18,9))
for i, patch in enumerate(patches[0]):
    w,h = int(np.ceil(IMG_SHAPE[0]/demo_patch_size)), int(np.ceil(IMG_SHAPE[1]/demo_patch_size))
    ax = plt.subplot(w,h,i+1)
    patch_img = tf.reshape(patch, (demo_patch_size, demo_patch_size, 3))
    plt.imshow(patch_img.numpy().astype("float32")/255.)
    plt.axis("off")
plt.tight_layout()
plt.subplots_adjust(wspace=0.03, hspace=0.06)
plt.show()

with tf.device('/CPU:0'):
    enc_patches_rescaled = patch_creator(tf.expand_dims(tf.image.resize(tf.reshape(tf.reduce_mean(encoded_patches[0], axis=-1), (w, h, 1)), (IMG_SHAPE[0], IMG_SHAPE[1])), axis=0))


print("\n\n\n... IMAGE EMBEDDING AS PATCH VISUALIZATION ...\n")
plt.figure(figsize=(18,9))
for i, enc_patch in enumerate(enc_patches_rescaled[0]):
    ax = plt.subplot(w,h,i+1)
    enc_patch_img = tf.reshape(enc_patch, (demo_patch_size, demo_patch_size))
    plt.imshow(enc_patch_img.numpy().astype("float32")/255., cmap="jet")
    plt.axis("off")
plt.tight_layout()
plt.subplots_adjust(wspace=0.03, hspace=0.06)
plt.show()

with tf.device('/CPU:0'):
    tmp_img = tf.image.resize(tf.reshape(tf.reduce_mean(demo_encoder_output[0], axis=-1), (w, h, 1)), (IMG_SHAPE[0], IMG_SHAPE[1]))
    tmp_img = tf.cast(255*((tmp_img-tf.math.reduce_min(tmp_img))/(tf.math.reduce_max(tmp_img)-tf.math.reduce_min(tmp_img))), dtype=tf.uint8)
    enc_output_patches_rescaled = patch_creator(tf.expand_dims(tmp_img, axis=0))

print("\n\n\n... ENCODER OUTPUT AS PATCH VISUALIZATION ...\n")
plt.figure(figsize=(18,9))
for i, enc_out_patch in enumerate(enc_output_patches_rescaled[0]):
    ax = plt.subplot(w,h,i+1)
    enc_out_patch_img = tf.reshape(enc_out_patch, (demo_patch_size, demo_patch_size))
    plt.imshow(enc_out_patch_img.numpy().astype("float32")/255., cmap="jet")
    plt.axis("off")
plt.tight_layout()
plt.subplots_adjust(wspace=0.03, hspace=0.06)
plt.show()

print("\n\n\n\n\tSUMMARY\n")
ViT.summary()

<h3 style="font-family: Verdana; font-size: 16px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.2.0 TRANSFORMER - HYPERPARAMETERS</h3>

---


In [ ]:
D_MODEL = IMG_EMB_DEPTH
N_PE_POS = 72
D_FF = 1024

print(f"\n... THE INPUT 'SEQUENCE LENGTH'                  IS {IMG_SEQ_LEN}  (output of image encoder - shape flattened) ...")
print(f"... THE INPUT 'EMBEDDING DEPTH'                  IS {IMG_EMB_DEPTH}  (output of image encoder - # of channels) ...")
print(f"... THE NUMBER OF POSITIONAL ENCODING POSITIONS  IS {N_PE_POS}   (arbitray) ...\n")

In [ ]:
def get_angles(pos, i, d_model):
    angle_rates = tf.constant(1, TARGET_DTYPE) / tf.math.pow(tf.constant(10000, TARGET_DTYPE), (tf.constant(2, dtype=TARGET_DTYPE) * tf.cast((i//2), TARGET_DTYPE))/d_model)
    return pos * angle_rates

def do_interleave(arr_a, arr_b):
    a_arr_tf_column = tf.range(arr_a.shape[1])*2 # [0 2 4 ...]
    b_arr_tf_column = tf.range(arr_b.shape[1])*2+1 # [1 3 5 ...]
    column_indices = tf.argsort(tf.concat([a_arr_tf_column,b_arr_tf_column],axis=-1))
    column, row = tf.meshgrid(column_indices,tf.range(arr_a.shape[0]))
    combine_indices = tf.stack([row,column],axis=-1)
    combine_value = tf.concat([arr_a,arr_b],axis=1)
    return tf.gather_nd(combine_value,combine_indices)

def positional_encoding_1d(position, d_model):
    angle_rads = get_angles(tf.cast(tf.range(position)[:, tf.newaxis], TARGET_DTYPE),
                            tf.cast(tf.range(d_model)[tf.newaxis, :], TARGET_DTYPE),
                            d_model)
    
    sin_angle_rads = tf.math.sin(angle_rads[:, ::2])
    cos_angle_rads = tf.math.cos(angle_rads[:, 1::2])
    angle_rads = do_interleave(sin_angle_rads, cos_angle_rads)
    pos_encoding = angle_rads[tf.newaxis, ...]
    return pos_encoding

def np_positional_encoding_2d(row,col,d_model):
    
    assert d_model % 2 == 0
    row_pos = np.repeat(np.arange(row),col)[:,np.newaxis]
    col_pos = np.repeat(np.expand_dims(np.arange(col),0),row,axis=0).reshape(-1,1)

    angle_rads_row = get_angles(row_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2).numpy()
    angle_rads_col = get_angles(col_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2).numpy()
    
    angle_rads_row[:, 0::2] = np.sin(angle_rads_row[:, 0::2])
    angle_rads_row[:, 1::2] = np.cos(angle_rads_row[:, 1::2])
    angle_rads_col[:, 0::2] = np.sin(angle_rads_col[:, 0::2])
    angle_rads_col[:, 1::2] = np.cos(angle_rads_col[:, 1::2])
    pos_encoding = np.concatenate([angle_rads_row,angle_rads_col],axis=1)[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=TARGET_DTYPE)

def positional_encoding_2d(row,col,d_model):
    
    row_pos = tf.repeat(tf.range(row), col)[:, tf.newaxis]
    col_pos = tf.reshape(tf.repeat(tf.expand_dims(tf.range(col),0), row, axis=0), (-1, 1))

    angle_rads_row = get_angles(tf.cast(row_pos, tf.float32), tf.range(d_model//2)[tf.newaxis,:], d_model//2)
    angle_rads_col = get_angles(tf.cast(col_pos, tf.float32), tf.range(d_model//2)[tf.newaxis,:], d_model//2)

    sin_angle_rads_row = tf.math.sin(angle_rads_row[:, ::2])
    cos_angle_rads_row = tf.math.cos(angle_rads_row[:, 1::2])
    angle_rads_row = do_interleave(sin_angle_rads_row, cos_angle_rads_row)

    sin_angle_rads_col = tf.math.sin(angle_rads_col[:, ::2])
    cos_angle_rads_col = tf.math.cos(angle_rads_col[:, 1::2])
    angle_rads_col = do_interleave(sin_angle_rads_col, cos_angle_rads_col)
    
    pos_encoding = tf.concat([angle_rads_row,angle_rads_col],axis=1)[tf.newaxis, ...]
    return pos_encoding

pos_encoding = positional_encoding_1d(256, 512)

print(pos_encoding.shape)

plt.figure(figsize=(6,4))
plt.pcolormesh(tf.cast(pos_encoding[0], tf.float32), cmap='RdBu')
plt.xlim((0, 512))
plt.ylim((0, 256))
plt.xlabel('Depth', fontweight="bold")
plt.ylabel('Position', fontweight="bold")
plt.title("Visualization of Positional Encoding", fontweight="bold")
plt.colorbar()
plt.show()

In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), TARGET_DTYPE)

    return seq[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return tf.cast(mask, TARGET_DTYPE)

def create_mask(inp, tar):

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
    return tf.cast(combined_mask, TARGET_DTYPE)

x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
print(f"x --\n{x}\n\n\nPADDING MASK\n{create_padding_mask(x)}\n")
print("")

x = tf.random.uniform((1, 5))
print(f"x.shape[1] -- {x.shape[1]}")
print(f"\n\nLOOK-AHEAD MASK\n{create_look_ahead_mask(x.shape[1])}\n")

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):

    matmul_qk = tf.matmul(q, k, transpose_b=True)  

    
    dk = tf.cast(tf.shape(k)[-1], TARGET_DTYPE)

    
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  

    output = tf.matmul(attention_weights, v)
    
    return output, attention_weights


def print_out(q, k, v):
    temp_out, temp_attn = scaled_dot_product_attention(q, k, v, None)

    print(f'Attention weights are:\n\t-->{temp_attn}')

    print(f'\nOutput is:\n\t-->{temp_out}')


np.set_printoptions(suppress=True)


temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=TARGET_DTYPE)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=TARGET_DTYPE)  # (4, 2)

print(f"\n-----------------------\n\nTEMP K:\n\n{temp_k}\n")
print(f"\n-----------------------\n\nTEMP V:\n\n{temp_v}\n")

temp_q = tf.constant([[0, 10, 0]], dtype=TARGET_DTYPE)  # (1, 3)
print(f"\n-----------------------\n\nTEMP Q:\n\n{temp_q} \n")
print_out(temp_q, temp_k, temp_v)

temp_q = tf.constant([[0, 0, 10]], dtype=TARGET_DTYPE)  # (1, 3)
print(f"\n-----------------------\n\nTEMP Q:\n\n{temp_q} \n")
print_out(temp_q, temp_k, temp_v)

temp_q = tf.constant([[10, 10, 0]], dtype=TARGET_DTYPE)  # (1, 3)
print(f"\n-----------------------\n\nTEMP Q:\n\n{temp_q} \n")
print_out(temp_q, temp_k, temp_v)

temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=TARGET_DTYPE)  # (3, 3)
print(f"\n-----------------------\n\nTEMP Q:\n\n{temp_q} \n")
print_out(temp_q, temp_k, temp_v)

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        
        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
        
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
    
        q = self.wq(q)  
    
        k = self.wk(k)  
    
        v = self.wv(v)  

    
        q = self.split_heads(q, batch_size)  
    
        k = self.split_heads(k, batch_size)  
    
        v = self.split_heads(v, batch_size)  
        
        scaled_attention, attention_weights = \
            scaled_dot_product_attention(q, k, v, mask)
    
        
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        
        concat_attention = \
            tf.reshape(scaled_attention, (batch_size, -1, self.d_model))  

        
        output = self.dense(concat_attention)  
            
        return output, attention_weights


temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512), dtype=TARGET_DTYPE)  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, y, y, mask=None)
print(f"Custom MHA Layer:\n\t-->{out[0,:2]}\n\t-->{(out.shape, attn.shape)}\n")

# TF NATIVE
temp_mha = tf.keras.layers.MultiHeadAttention(8, 512)
out, attn = temp_mha(y, y, y, attention_mask=None, return_attention_scores=True)
print(f"TF MHA Layer:\n\t-->{out[0,:2]}\n\t-->{(out.shape, attn.shape)}\n")

del temp_mha

<h3 style="font-family: Verdana; font-size: 16px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.2.5 TRANSFORMER - POINT-WISE FEED FORWARD NEURAL NETWORK</h3>

---

Point wise feed forward network consists of two fully-connected layers with a ReLU activation in-between.

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        
        tf.keras.layers.Dense(dff, activation='relu'),  
        
        tf.keras.layers.Dense(d_model)  
    ])

sample_ffn = point_wise_feed_forward_network(512, D_FF)
print("\nFFN INPUT & OUTPUT SHAPE: " \
      f"{sample_ffn(tf.random.uniform((64, 50, 512), dtype=TARGET_DTYPE)).shape}" \
      "\n\nFFN SUMMARY:")
print(sample_ffn.summary())

del sample_ffn

In [ ]:
class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        
        super(TransformerEncoderLayer, self).__init__()

        self.mha = tf.keras.layers.MultiHeadAttention(num_heads, key_dim=d_model,)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, x, training, mask=None):
        
        attn_output, _ = self.mha(x, x, x, mask, return_attention_scores=True) 

        attn_output = self.dropout1(attn_output, training=training)
        
        out1 = self.layernorm1(x + attn_output, training=training)  
        
        ffn_output = self.ffn(out1, training=training)  
        ffn_output = self.dropout2(ffn_output, training=training)
        
        out2 = self.layernorm2(out1 + ffn_output, training=training)  
        
        return out2

sample_encoder_layer = TransformerEncoderLayer(D_MODEL, 8, D_FF)
sample_encoder_layer_output = sample_encoder_layer(demo_encoder_output, training=False, mask=None)
del sample_encoder_layer

# (batch_size, input_seq_len, d_model)
sample_encoder_layer_output.shape  

In [ ]:
class TransformerDecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):

        super(TransformerDecoderLayer, self).__init__()

        # WE COULD USE A CUSTOM DEFINED MHA MODEL BUT WE WILL USE TFA INSTEAD
        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)
        
        self.ffn = point_wise_feed_forward_network(d_model, dff)
    
        # Layer Normalization Layers
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        # Dropout Layers
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout3 = tf.keras.layers.Dropout(dropout_rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask=None, padding_mask=None):
        
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)

        out1 = self.layernorm1(attn1 + x, training=training)
    
       
        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask) 
        attn2 = self.dropout2(attn2, training=training)
        
        # Residual connection followed by layer normalization
        #   – (batch_size, target_seq_len, d_model)
        out2 = self.layernorm2(attn2 + out1, training=training)  
        
        # (batch_size, target_seq_len, d_model)
        ffn_output = self.ffn(out2, training=training)  
        ffn_output = self.dropout3(ffn_output, training=training)

        # Residual connection followed by layer normalization
        #   – (batch_size, target_seq_len, d_model)
        out3 = self.layernorm3(ffn_output + out2, training=training)  
        
        return out3, attn_weights_block1, attn_weights_block2

sample_decoder_layer = TransformerDecoderLayer(D_MODEL, 8, D_FF)
sample_decoder_layer_output, _, _ = sample_decoder_layer(tf.random.uniform((BATCH_SIZE_DEBUG, MAX_LEN, D_MODEL), dtype=TARGET_DTYPE), sample_encoder_layer_output, False, None, None)
del sample_decoder_layer

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)

In [ ]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 maximum_position_encoding, dropout_rate=0.1):
        super(TransformerEncoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.pos_encoding = positional_encoding_1d(maximum_position_encoding, self.d_model)
        self.enc_layers = [TransformerEncoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        
    def call(self, x, training, mask=None):
        

        # adding embedding and position encoding.
        #   – (batch_size, input_seq_len, d_model)
        x += self.pos_encoding
        x = self.dropout(x, training=training)
        
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        #   – (batch_size, input_seq_len, d_model)
        return x  

sample_encoder = TransformerEncoder(num_layers=2, 
                         d_model=D_MODEL, 
                         num_heads=8, 
                         dff=D_FF,
                         maximum_position_encoding=IMG_SEQ_LEN)

sample_encoder_output = sample_encoder(demo_encoder_output, training=False, mask=None)
del sample_encoder

print(sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

In [ ]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super(TransformerDecoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        
        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding_1d(maximum_position_encoding, d_model)
        
        self.dec_layers = [TransformerDecoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, x, enc_output, training, look_ahead_mask=None, padding_mask=None):

        seq_len = tf.shape(x)[1]
        attention_weights = {}
        
        # adding embedding and position encoding.
        #   – (batch_size, target_seq_len, d_model)
        x = self.embedding(x)  
        x *= tf.math.sqrt(tf.cast(self.d_model, TARGET_DTYPE))
        x += self.pos_encoding[:, :seq_len, :]
        
        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)
            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights


sample_decoder = TransformerDecoder(num_layers=2, d_model=D_MODEL, num_heads=8, 
                         dff=D_FF, target_vocab_size=VOCAB_LEN,
                         maximum_position_encoding=MAX_LEN)
temp_input = tf.random.uniform((1, MAX_LEN), dtype=tf.int64, minval=0, maxval=VOCAB_LEN)
output, attn = sample_decoder(temp_input, 
                              enc_output=demo_encoder_output, 
                              training=False,
                              look_ahead_mask=None, 
                              padding_mask=None)
del sample_decoder
output.shape, attn['decoder_layer2_block2'].shape

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 target_vocab_size, pe_input, pe_target, dropout_rate=0.1):
        """TBD"""
        super(Transformer, self).__init__()
        
        self.t_encoder = TransformerEncoder(num_layers, d_model, num_heads, dff, pe_input, dropout_rate)
        self.t_decoder = TransformerDecoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, dropout_rate)
        self.t_final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, t_inp, t_tar, 
             training, enc_padding_mask=None, 
             look_ahead_mask=None, dec_padding_mask=None):
        """TBD"""
        # (batch_size, inp_seq_len, d_model)
        enc_output = self.t_encoder(t_inp, training, enc_padding_mask)  
        
        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        dec_output, attention_weights = self.t_decoder(t_tar, 
                                                       enc_output, 
                                                       training, 
                                                       look_ahead_mask, 
                                                       dec_padding_mask)

        # (batch_size, tar_seq_len, target_vocab_size)
        final_output = self.t_final_layer(dec_output)  
    
        return final_output, attention_weights



<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.3 CREATE A LEARNING RATE SCHEDULER</h3>

---

We utiliize the learning rate scheduler from the "Attention Is All You Need" paper with some minor tweaks.

In [ ]:
print("\n... LEARNING RATE SCHEDULE CREATION STARTING ...\n")

# Part of the Training Configuration
EPOCHS = 30
TOTAL_STEPS = TRAIN_STEPS*EPOCHS

# Learning Rate Scheduler Configuration
WARM_STEPS = (TRAIN_STEPS-1)*4 # Suuuuuper long ramp-up

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model*1.75) * tf.math.minimum(arg1, arg2)
    
temp_learning_rate_schedule = CustomSchedule(D_MODEL, WARM_STEPS)
plt.plot(temp_learning_rate_schedule(tf.range(TRAIN_STEPS*EPOCHS, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")
plt.show()


print("\n... LEARNING RATE SCHEDULE CREATION FINISHED ...\n")

# MODEL TRAINING

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">6.1 INDIVIDUAL TRAIN STEP</h3>

---

INFORMATION

In [ ]:
def train_step(_image_batch, _inchi_batch):
    """ Forward pass (calculate gradients)
    
    Args:
        _image_batch (): TBD
        _inchi_batch (): TBD
    
    Returns:
        tbd
    """
    _inchi_batch_input  = _inchi_batch[:, :-1]
    _inchi_batch_target = _inchi_batch[:, 1:]
    combined_mask = create_mask(_inchi_batch_input, _inchi_batch_target)

    with tf.GradientTape() as tape:
        _image_embedding = encoder(_image_batch, training=True)
        prediction_batch, _ = transformer(_image_embedding, _inchi_batch_input, training=True, look_ahead_mask=combined_mask)
        
        # Update Loss Accumulator
        batch_loss = loss_fn(_inchi_batch_target, prediction_batch)/(MAX_LEN-1)

        # Update Accuracy Metric
        metrics["train_acc"].update_state(_inchi_batch_target, prediction_batch, 
                                          sample_weight=tf.where(tf.not_equal(_inchi_batch_target, PAD_TOKEN), 1.0, 0.0))


    # backpropagation using variables, gradients and loss
    #    - split this into two seperate optimizers/lrs/etc in the future
    #    - we use the batch loss accumulation to update gradients
    gradients = tape.gradient(batch_loss, encoder.trainable_variables + transformer.trainable_variables)
    gradients, _ = tf.clip_by_global_norm(gradients, 10.0)
    optimizer.apply_gradients(zip(gradients, encoder.trainable_variables+transformer.trainable_variables))
    
    metrics["batch_loss"].update_state(batch_loss)
    metrics["train_loss"].update_state(batch_loss)

@tf.function
def dist_train_step(_image_batch, _inchi_batch):
    strategy.run(train_step, args=(_image_batch, _inchi_batch))

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">6.2 INDIVIDUAL VAL STEP</h3>

---

INFORMATION

In [ ]:
def val_step(_image_batch, _inchi_batch):
    """ Forward pass (calculate gradients)
    
    Args:
        image_batch (): TBD
        inchi_batch (): TBD
    
    Returns:
        tbd
    """
    
    # Initialize batch_loss
    batch_loss = tf.constant(0.0, TARGET_DTYPE)       
    transformer_pred_batch = tf.ones((REPLICA_BATCH_SIZE, 1), dtype=tf.uint8)
    
    # Get image embedding (once)
    _image_embedding = encoder(_image_batch, training=False)
    
    # Teacher forcing - feeding the target as the next input
    for c_idx in range(1, MAX_LEN):
        gt_batch_id = _inchi_batch[:, c_idx]
        combined_mask = create_mask(_inchi_batch, transformer_pred_batch)
        
        # predictions.shape == (batch_size, seq_len, vocab_size)
        prediction_batch, attention_weights = transformer(_image_embedding, transformer_pred_batch, training=False, look_ahead_mask=combined_mask)
        predicted_batch_id = prediction_batch[:, -1:, :]
        
        # Update Loss Accumulator
        batch_loss += loss_fn(gt_batch_id, predicted_batch_id[:, -1])
    
        # Update Accuracy Metric
        metrics["val_acc"].update_state(gt_batch_id, predicted_batch_id[:, -1],
                                        sample_weight=tf.where(tf.not_equal(gt_batch_id, PAD_TOKEN), 1.0, 0.0))

        # no teacher forcing, predicted char is next transformer input
        transformer_pred_batch = tf.concat([transformer_pred_batch, tf.cast(tf.argmax(predicted_batch_id, axis=-1), tf.uint8)], axis=-1)
        
    # Update Loss Metric
    metrics["val_loss"].update_state(batch_loss)
    return transformer_pred_batch    

    
@tf.function
def dist_val_step(_val_dist_ds):
    _val_image_batch, _val_inchi_batch = next(_val_dist_ds)
    predictions_seq_batch_per_replica = strategy.run(val_step, args=(_val_image_batch, _val_inchi_batch))
    predictions_seq_batch_accum = strategy.gather(predictions_seq_batch_per_replica, axis=0)
    _val_inchi_batch_accum = strategy.gather(_val_inchi_batch, axis=0)
    return predictions_seq_batch_accum, _val_inchi_batch_accum

#INFER ON TEST DATA

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">7.1 INDIVIDUAL TEST STEP (AND DISTRIBUTED)</h3>

---

INFORMATION

In [ ]:
def test_step(_image_batch):
    
    
    transformer_pred_batch = tf.ones((REPLICA_BATCH_SIZE, 1), dtype=tf.uint8)
    
    # Get image embedding (once)
    _image_embedding = encoder(_image_batch, training=False)
    
    # Teacher forcing - feeding the target as the next input
    for c_idx in range(1, MAX_LEN):
        
        combined_mask = create_mask(None, transformer_pred_batch)
        
        # predictions.shape == (batch_size, seq_len, vocab_size)
        prediction_batch, attention_weights = transformer(_image_embedding, transformer_pred_batch, training=False, look_ahead_mask=combined_mask)
        predicted_batch_id = prediction_batch[:, -1:, :]
        
        # no teacher forcing, predicted char is next transformer input
        transformer_pred_batch = tf.concat([transformer_pred_batch, tf.cast(tf.argmax(predicted_batch_id, axis=-1), tf.uint8)], axis=-1)
        
    return transformer_pred_batch 

    
@tf.function
def distributed_test_step(_img_batch, _img_ids):
    per_replica_seqs = strategy.run(test_step, args=(_img_batch,))
    predictions = strategy.gather(per_replica_seqs, axis=0)
    pred_ids = strategy.gather(_img_ids, axis=0)
    return predictions, pred_ids


@tf.function
def distributed_test_step_v2(_test_dist_ds):
    _test_image_batch, _test_id_batch = next(_test_dist_ds)
    predictions_seq_batch_per_replica = strategy.run(test_step, args=(_test_image_batch,))
    predictions_seq_batch_accum = strategy.gather(predictions_seq_batch_per_replica, axis=0)
    _test_id_batch_accum = strategy.gather(_test_id_batch, axis=0)
    return predictions_seq_batch_accum, _test_id_batch_accum

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">7.2 RAW INFERENCE LOOP</h3>

---

INFORMATION

In [ ]:
# To Store The Preds
all_pred_arr = tf.zeros((1, MAX_LEN), dtype=tf.uint8)
all_pred_ids = tf.zeros((1, 1), dtype=tf.string)

# Create an iterator
dist_test_ds = iter(strategy.experimental_distribute_dataset(test_ds))
for i in tqdm(range(TEST_STEPS), total=TEST_STEPS): 
    preds, pred_ids = distributed_test_step_v2(dist_test_ds)
    all_pred_arr = tf.concat([all_pred_arr, preds], axis=0)
    all_pred_ids = tf.concat([all_pred_ids, tf.expand_dims(pred_ids, axis=-1)], axis=0)

In [ ]:
# To Store The Preds
all_pred_arr = tf.zeros((1, MAX_LEN), dtype=tf.uint8)
all_pred_ids = tf.zeros((1, 1), dtype=tf.string)

# Create an iterator
dist_test_ds = iter(strategy.experimental_distribute_dataset(test_ds))
for i in tqdm(range(TEST_STEPS), total=TEST_STEPS): 
    img_batch, id_batch = next(dist_test_ds)
    preds, pred_ids = distributed_test_step(img_batch, id_batch)
    all_pred_arr = tf.concat([all_pred_arr, preds], axis=0)
    all_pred_ids = tf.concat([all_pred_ids, tf.expand_dims(pred_ids, axis=-1)], axis=0)

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">7.3 TEST PRED POST-PROCESSING</h3>

---

INFORMATION

In [ ]:
def arr_2_inchi(arr):
    inchi_str = ''
    for i in arr:
        c = int_2_tok.get(i)
        if c=="<END>":
            break
        inchi_str += c
    return inchi_str

pred_df = pd.DataFrame({
    "image_id":[x[0].decode() for x in tqdm(all_pred_ids[1:-REQUIRED_DATASET_PAD].numpy(), total=N_TEST)], 
    "InChI":[arr_2_inchi(pred_arr) for pred_arr in tqdm(all_pred_arr[1:-REQUIRED_DATASET_PAD].numpy(), total=N_TEST)]
})

pred_df = pred_df.sort_values(by="image_id").reset_index(drop=True)
pred_df

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">7.4 SAVE SUBMISSION.CSV</h3>

In [ ]:
pred_df.to_csv("submission.csv", index=False)